__DATA CLEANING__

In [5]:
import re
import html
from bs4 import BeautifulSoup
import pandas as pd

In [7]:
df = pd.read_csv("../data/paycell_reviews_play_store.csv")
df

,Yorum,Puan,Tarih
0,şuan uygulama da sorunlar var bankadan karta p...,1,2025-07-07 10:25:20
1,bir türlü decont alamıyoruz. bazı yerler ödeme...,1,2025-07-07 09:08:45
2,bu nasıl bir uygulama bir haftadır açmaya çalı...,1,2025-07-06 22:22:35
3,yabancı olduğum için doğrulama yapamıyorum,1,2025-07-06 20:02:06
4,şifre güncelle diyorsunuz ama guncelletmiuorsu...,3,2025-07-06 18:34:42
...,...,...,...
68634,Çok güzel,5,2017-08-25 10:43:11
68635,Mobil ödeme bakiye ve islem tarihcesi goruntul...,5,2017-08-19 15:08:30
68636,Beklemedeyiz,4,2017-08-17 06:38:45
68637,Fine hit :),5,2017-08-16 20:23:34


In [8]:
def clean_text(text):
    if pd.isnull(text):
        return ""

    # for html
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = html.unescape(text)

    text = re.sub(r'http\S+|www.\S+', '', text) 
    text = re.sub(r'[^\w\sçğıöşüÇĞİÖŞÜ]', '', text, flags=re.UNICODE)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()

    return text



In [10]:
df['Temiz_Yorum'] = df['Yorum'].apply(clean_text)
df[['Yorum', 'Temiz_Yorum']]

,Yorum,Temiz_Yorum
0,şuan uygulama da sorunlar var bankadan karta p...,şuan uygulama da sorunlar var bankadan karta p...
1,bir türlü decont alamıyoruz. bazı yerler ödeme...,bir türlü decont alamıyoruz bazı yerler ödeme ...
2,bu nasıl bir uygulama bir haftadır açmaya çalı...,bu nasıl bir uygulama bir haftadır açmaya çalı...
3,yabancı olduğum için doğrulama yapamıyorum,yabancı olduğum için doğrulama yapamıyorum
4,şifre güncelle diyorsunuz ama guncelletmiuorsu...,şifre güncelle diyorsunuz ama guncelletmiuorsu...
...,...,...
68634,Çok güzel,çok güzel
68635,Mobil ödeme bakiye ve islem tarihcesi goruntul...,mobil ödeme bakiye ve islem tarihcesi goruntul...
68636,Beklemedeyiz,beklemedeyiz
68637,Fine hit :),fine hit


__PREPROCESSING__

In [56]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import string

In [57]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\ismet
[nltk_data]     bal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [60]:
stop_words = set(stopwords.words("turkish"))

def remove_stopwords(text):
    tokens = text.split() 
    filtered_tokens = [word.lower() for word in tokens 
                       if word.lower() not in stop_words and
                       word not in string.punctuation and
                       len(word) > 2]
    return filtered_tokens

df = df[df["Temiz_Yorum"].notnull()]
df["tokens"] = df["Temiz_Yorum"].astype(str).apply(simple_tokenize_and_remove_stopwords)
df[["Yorum", "Temiz_Yorum", "tokens"]]


,Yorum,Temiz_Yorum,tokens
0,şuan uygulama da sorunlar var bankadan karta p...,şuan uygulama da sorunlar var bankadan karta p...,"[şuan, uygulama, sorunlar, var, bankadan, kart..."
1,bir türlü decont alamıyoruz. bazı yerler ödeme...,bir türlü decont alamıyoruz bazı yerler ödeme ...,"[bir, türlü, decont, alamıyoruz, yerler, ödeme..."
2,bu nasıl bir uygulama bir haftadır açmaya çalı...,bu nasıl bir uygulama bir haftadır açmaya çalı...,"[bir, uygulama, bir, haftadır, açmaya, çalışıy..."
3,yabancı olduğum için doğrulama yapamıyorum,yabancı olduğum için doğrulama yapamıyorum,"[yabancı, olduğum, doğrulama, yapamıyorum]"
4,şifre güncelle diyorsunuz ama guncelletmiuorsu...,şifre güncelle diyorsunuz ama guncelletmiuorsu...,"[şifre, güncelle, diyorsunuz, guncelletmiuorsu..."
...,...,...,...
68634,Çok güzel,çok güzel,[güzel]
68635,Mobil ödeme bakiye ve islem tarihcesi goruntul...,mobil ödeme bakiye ve islem tarihcesi goruntul...,"[mobil, ödeme, bakiye, islem, tarihcesi, gorun..."
68636,Beklemedeyiz,beklemedeyiz,[beklemedeyiz]
68637,Fine hit :),fine hit,"[fine, hit]"
